<a href="https://colab.research.google.com/github/chiamaka249/IgboNER/blob/main/Practice_of_Copy_of_igbobert4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile
import os
from google.colab import drive

drive.mount('/content/gdrive')

def unzip(zipfilename):
  try:
    with zipfile.ZipFile(zipfilename, 'r') as zip_ref:
      zip_ref.extractall(zipfilename[:-4])
      return f"'{zipfilename}' unzipped!"
  except FileNotFoundError:
    print(f"Cannot find '{zipfilename}' file")

In [ ]:
# We won't need TensorFlow here
!pip uninstall -y tensorflow
# Install `transformers` from master
!pip install git+https://github.com/huggingface/transformers
!pip list | grep -E 'transformers|tokenizers'
# transformers version at notebook update --- 2.11.0
# tokenizers version at notebook update --- 0.8.0rc1

In [ ]:
from tokenizers import ByteLevelBPETokenizer
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./igbo_bert4/vocab.json",
    "./igbo_bert4/merges.txt",
)

tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=512)

In [ ]:
unzip('/content/gdrive/MyDrive/igbo_bert/igbo_bert3.zip')

"'/content/gdrive/MyDrive/igbo_bert/igbo_bert3.zip' unzipped!"

In [ ]:
working_dir='/content/gdrive/MyDrive/igbo_bert/igbo_bert3'

In [ ]:
os.listdir(working_dir)

['checkpoint-10000',
 'runs',
 'config.json',
 'pytorch_model.bin',
 'vocab.json',
 'merges.txt',
 'training_args.bin']

In [ ]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained(working_dir, max_len=512)

In [ ]:
tokenizer.encode("Simone gara ụka ụnyahụ guọ egwu ma ga-kwa taa.")

[0, 39426, 755, 1123, 7384, 1367, 265, 686, 313, 311, 17, 315, 837, 18, 2]

In [ ]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)

In [ ]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

In [ ]:
model.num_parameters()
# => 83 million parameters

83504416

In [ ]:
model.load_state_dict(torch.load(os.path.join(working_dir, 'pytorch_model.bin')))

In [ ]:
pip install pytorch_pretrained_bert

     |████████████████████████████████| 123 kB 9.9 MB/s 
     |████████████████████████████████| 131 kB 79.3 MB/s 
     |████████████████████████████████| 8.4 MB 54.5 MB/s 
     |████████████████████████████████| 79 kB 8.9 MB/s 
     |████████████████████████████████| 138 kB 78.1 MB/s 
     |████████████████████████████████| 127 kB 75.0 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, which is not installed.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
trainer.save_model("./igbo_bert4")

Saving model checkpoint to ./igbo_bert4
Configuration saved in ./igbo_bert4/config.json
Model weights saved in ./igbo_bert4/pytorch_model.bin


In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./igbo_bert4",
    tokenizer="./igbo_bert4"
)

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.13.0.dev0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 52000
}

loading configuration file ./igbo_bert4/config.json
Model config RobertaConfig {
  "_name_or_path": "./igbo_bert4",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dro

OSError: ignored

In [ ]:
# The sun <mask>.
# =>

fill_mask("Abụ m Maazị <mask>.") #= okafor/Ọkafọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.013978319242596626,
  'sequence': 'Abụ m Maazị Ọkafọ.',
  'token': 5775,
  'token_str': ' Ọkafọ'},
 {'score': 0.010930763557553291,
  'sequence': 'Abụ m Maazị O.',
  'token': 378,
  'token_str': ' O'},
 {'score': 0.01065503153949976,
  'sequence': 'Abụ m Maazị Obianọ.',
  'token': 1566,
  'token_str': ' Obianọ'},
 {'score': 0.00977820809930563,
  'sequence': 'Abụ m Maazị M.',
  'token': 398,
  'token_str': ' M'},
 {'score': 0.0066724903881549835,
  'sequence': 'Abụ m Maazị E.',
  'token': 426,
  'token_str': ' E'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na <mask> ji na akara.") #= eri
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.12062991410493851,
  'sequence': 'Nwaanyị na ya ji na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.04657674953341484,
  'sequence': 'Nwaanyị na nwunye ji na akara.',
  'token': 740,
  'token_str': ' nwunye'},
 {'score': 0.023468855768442154,
  'sequence': 'Nwaanyị na nna ji na akara.',
  'token': 723,
  'token_str': ' nna'},
 {'score': 0.02191004902124405,
  'sequence': 'Nwaanyị na ha ji na akara.',
  'token': 297,
  'token_str': ' ha'},
 {'score': 0.021505558863282204,
  'sequence': 'Nwaanyị na nwaanyị ji na akara.',
  'token': 629,
  'token_str': ' nwaanyị'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Chineke ga- ebibikwa ndị niile na- eme ihe <mask>.") #=ọjọọ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.3927464187145233,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọjọọ.',
  'token': 726,
  'token_str': ' ọjọọ'},
 {'score': 0.13551250100135803,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ọma.',
  'token': 503,
  'token_str': ' ọma'},
 {'score': 0.06629031151533127,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe a.',
  'token': 266,
  'token_str': ' a'},
 {'score': 0.06092679873108864,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe niile.',
  'token': 434,
  'token_str': ' niile'},
 {'score': 0.031389206647872925,
  'sequence': 'Chineke ga- ebibikwa ndị niile na- eme ihe ike.',
  'token': 363,
  'token_str': ' ike'}]

In [ ]:
fill_mask("ọba akwụkwọ Ọkammụta Kenneth Dike dị <mask>.") #n'Awka

# This is the beginning of a beautiful <mask>.
# =>

[{'score': 0.14023567736148834,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị iche.',
  'token': 447,
  'token_str': ' iche'},
 {'score': 0.07257493585348129,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị icheiche.',
  'token': 3871,
  'token_str': ' icheiche'},
 {'score': 0.07037945091724396,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị nso.',
  'token': 606,
  'token_str': ' nso'},
 {'score': 0.042156368494033813,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị ugbua.',
  'token': 1350,
  'token_str': ' ugbua'},
 {'score': 0.04174605384469032,
  'sequence': 'ọba akwụkwọ Ọkammụta Kenneth Dike dị egwu.',
  'token': 703,
  'token_str': ' egwu'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Nwaanyị na eri <mask> na akara.") #= ji
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.13028940558433533,
  'sequence': 'Nwaanyị na eri ya na akara.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.05488795042037964,
  'sequence': 'Nwaanyị na eri nri na akara.',
  'token': 885,
  'token_str': ' nri'},
 {'score': 0.02480449341237545,
  'sequence': 'Nwaanyị na eri ihe na akara.',
  'token': 304,
  'token_str': ' ihe'},
 {'score': 0.019163204357028008,
  'sequence': 'Nwaanyị na eri ọkụ na akara.',
  'token': 707,
  'token_str': ' ọkụ'},
 {'score': 0.017493773251771927,
  'sequence': 'Nwaanyị na eri m na akara.',
  'token': 268,
  'token_str': ' m'}]

In [ ]:
# The sun <mask>.
# =>

fill_mask("Ọ bụ <mask>a ka a na- arịa .") #= mmadụ
# fill_mask("Nwaanyị na <mask> ji na akara.") #=eri

[{'score': 0.22005508840084076,
  'sequence': 'Ọ bụ-a ka a na- arịa.',
  'token': 17,
  'token_str': '-'},
 {'score': 0.019461117684841156,
  'sequence': 'Ọ bụ Nwannaa ka a na- arịa.',
  'token': 1546,
  'token_str': ' Nwanna'},
 {'score': 0.008462444879114628,
  'sequence': 'Ọ bụ ‘a ka a na- arịa.',
  'token': 531,
  'token_str': ' ‘'},
 {'score': 0.0064787482842803,
  'sequence': 'Ọ bụ yaa ka a na- arịa.',
  'token': 290,
  'token_str': ' ya'},
 {'score': 0.006270089186728001,
  'sequence': 'Ọ bụ ụmụa ka a na- arịa.',
  'token': 428,
  'token_str': ' ụmụ'}]

In [ ]:
import shutil

In [ ]:
shutil.make_archive("/content/igbo_bert4", 'zip', "igbo_bert4")

'/content/igbo_bert4.zip'

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
model_save_name = '/igbo_bert4.zip'
path = F"/content/gdrive/My Drive/{model_save_name}"
torch.save(model.state_dict(), path)

In [ ]:
from google.colab import files
files.download("/content/igbo_bert4.zip")

FileNotFoundError: ignored